In [1]:
import pandas as pd 
import yfinance as yf
from nltk.corpus import stopwords 
from nltk import download

In [2]:
import spacy
nlp=spacy.load("en_core_web_sm")

C:\Users\nares\anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\nares\anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\nares\anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\nares\anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\nares\anaconda3\lib\site-packages\spacy\util.py:887: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.5.4). If you see errors or degraded perf

In [3]:
import gensim.downloader as api
model = api.load('word2vec-google-news-300')

In [88]:
tickers=pd.read_excel(r"C:/Users/nares/Documents/chatbot/Ticker.xlsx")
data=pd.read_csv(r"C:/Users/nares/Documents/chatbot/questions.csv")
services=pd.read_csv(r"C:/Users/nares/Documents/chatbot/Services.csv")

In [58]:
services_list=['balance sheet','dividend','capital gains','cash flow','financial statement','income statement'
               ,'information','info','mutual fund holders','isin','ticker','holders','splits','history'
               ,'share price','stock price']

In [53]:
def identify_service(question):
    for i,service in enumerate(services_list):
        if service.lower() in question.lower():
            identify_company(question,services[services['details']==service]['code'].values[0])
            break
        elif i==len(services_list)D:
            print("Bot :we are not supporting this service,kindly try again")

In [12]:
def identify_company(question,req_service):
    companyname=[]
    distance=[]
    doc=nlp(question)
    companyname=[entity.text for entity in doc.ents if entity.label_=="ORG"]
    if len(companyname)!=0 and type(companyname[0])==str:
        companyname=companyname[0]
        if companyname.lower() in tickers['code'].values:
            ticker=tickers[tickers['code']==companyname.lower()]['symbol'].values[0]
            financial_data=financial_detail(req_service,ticker)
            print(financial_data)
        else:
            for i in tickers['company_name']:
                distance.append(model.wmdistance(companyname.lower(),i))
            tickers['dist']=distance
            tickers.sort_values(by=['dist'],inplace=True,ignore_index=True)
            if tickers['dist'][0]>=0.5:
                print("Bot :sorry,we could not find this company,we provide support only for NSE50 listed companies")
            else:
                ticker=tickers['symbol'][0]
                financial_detail(req_service,ticker)
    else:
        print("could you please provide company name(caps) along with required report name ? ")

In [86]:
def financial_detail(req_service,ticker):
    company=yf.Ticker(ticker)
    if req_service.startswith('get_'):
        financial_data_method = getattr(company, req_service)
        financial_data = financial_data_method()
        print(f"Bot: Please find the below")
        return financial_data
    elif req_service.startswith('current'):
        financial_data_method = getattr(company,'info')
        financial_data = financial_data_method['currentPrice']
        return (f"Bot: latest share price is {financial_data}")
    else:
        financial_data=getattr(company, req_service)
        print(f"Bot : please find the below")
        return financial_data

In [8]:
def financial_chatbot():
    import warnings
    warnings.filterwarnings('ignore')
    print("*****Type stop to end the chat*****")
    question=input('You : ')
    while question.lower()!="stop":
        if question=="bye":
            print('Bot : Bye')
            break
        dist=[]
        for j in data.Question:
            dist.append(model.wmdistance(question.lower(),j))
        data['dist']=dist
        data.sort_values(by=['dist'],inplace=True,ignore_index=True)
        if  data['dist'][0]>=0.5:
            print("Bot :sorry,could you please re-phrase your question or, \n     please try someother question")
        elif data['Answer'][0]=="Bye":
            print('Bot : Bye')
            break
        elif data['Answer'][0]=="Call_function":
            identify_service(question)
        else:
            print('Bot :',data['Answer'][0])
        question=input('You : ')

In [ ]:
financial_chatbot()

*****Type stop to end the chat*****
You : hI
Bot : Hello
You : how are you
Bot : I am doing good, and you?
You : yes i am good
Bot : How can I help you
You : how can you help me
Bot : I am Financial Chatbot created to provide stock market details of NSE listed companies.
You : can you provide the share price of HCL
Bot: latest share price is 1234.8
You : What is the isin of ITC
Bot: Please find the below
SG1C67001091
You : Can you provide the dividend list of ITC
Bot : please find the below
Date
1996-07-08 00:00:00+05:30     0.055556
1997-08-04 00:00:00+05:30     0.022222
1998-06-22 00:00:00+05:30     0.100000
1999-06-24 00:00:00+05:30     0.122222
2000-06-27 00:00:00+05:30     0.166667
2001-07-03 00:00:00+05:30     0.222222
2002-07-12 00:00:00+05:30     0.300000
2003-07-14 00:00:00+05:30     0.333333
2004-07-19 00:00:00+05:30     0.444444
2005-07-18 00:00:00+05:30     0.688889
2006-07-10 00:00:00+05:30     0.883333
2007-07-16 00:00:00+05:30     1.033333
2008-07-16 00:00:00+05:30     1